# 5. Deep Learning Computation

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))

X = torch.rand(2,20)
net(X)

tensor([[-0.1458,  0.0250, -0.2108, -0.1978,  0.1325, -0.1382,  0.0278, -0.0589,
         -0.0841,  0.0289],
        [-0.1879,  0.1257, -0.3135, -0.1036,  0.1113, -0.2336, -0.0608, -0.0618,
         -0.0059,  0.0177]], grad_fn=<AddmmBackward0>)

### 5.1.1. A Custom Block

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[-0.0491, -0.1584, -0.1300, -0.2776, -0.1341,  0.1627, -0.1304, -0.0175,
          0.1158, -0.1756],
        [-0.1472, -0.1691,  0.0883, -0.1427, -0.0569,  0.1177, -0.2196, -0.1219,
         -0.0422, -0.0987]], grad_fn=<AddmmBackward0>)

### 5.1.2. The Sequential Block

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X) :
        for block in self._modules.values():
            X = block(X)
        return X

In [7]:
net = MySequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))
net(X)

tensor([[-0.0408, -0.3139,  0.0760,  0.1291, -0.2317, -0.0815,  0.0632, -0.1894,
         -0.0377,  0.1309],
        [-0.0399, -0.2113,  0.1453,  0.0377, -0.1164,  0.0007,  0.1227, -0.1501,
         -0.1175,  0.1439]], grad_fn=<AddmmBackward0>)

### 5.1.3. Executing code in the Forward Propagation Function

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        self.linear = nn.Linear(20,20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)

        while X.abs().sum() > 1:
            X /= 2
        
        return X.sum()

In [9]:
net = FixedHiddenMLP()
net(X)

tensor(0.1796, grad_fn=<SumBackward0>)

In [12]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64), nn.ReLU(),
                                nn.Linear(64,32), nn.ReLU())
        self.linear = nn.Linear(32,16)

    def forward(self, X):
        return self.linear(self.net(X))

In [13]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16,20), FixedHiddenMLP())
chimera(X)

tensor(0.2035, grad_fn=<SumBackward0>)

## 5.2. Parameter Management

In [14]:
net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8,1))
X = torch.rand(size=(2,4))
net(X)

tensor([[-0.5047],
        [-0.4280]], grad_fn=<AddmmBackward0>)

### 5.2.1. Parameter Access

In [15]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.1446, -0.3304, -0.1958,  0.0344, -0.3269, -0.2057,  0.3110, -0.0711]])), ('bias', tensor([-0.1659]))])


In [16]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1659], requires_grad=True)
tensor([-0.1659])


In [17]:
net[2].weight.grad == None

True

#### All Parameters at Once

In [18]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [19]:
net.state_dict()['2.bias'].data

tensor([-0.1659])

#### Collecting Parameters from Nested Blocks

In [20]:
def block1():
    return nn.Sequential(nn.Linear(4,8), nn.ReLU(),
                         nn.Linear(8,4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())

    return net

In [21]:
rgnet = nn.Sequential(block2(), nn.Linear(4,1))
rgnet(X)

tensor([[0.4653],
        [0.4653]], grad_fn=<AddmmBackward0>)

In [22]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [23]:
rgnet[0][1][0].bias.data

tensor([ 0.0806,  0.2692,  0.4314,  0.3728, -0.2350, -0.4780, -0.0694,  0.2538])

### 5.2.2. Parameter Initialization

#### Built-in Initialization

In [24]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0239, -0.0162,  0.0170, -0.0027]), tensor(0.))

In [25]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [26]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.0344, -0.1866, -0.4948, -0.5578])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


#### Custom Initialization

In [29]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  8.6202,  7.5495, -0.0000],
        [ 0.0000, -0.0000,  7.1023, -5.9338]], grad_fn=<SliceBackward0>)

In [30]:
net[0].weight.data[:] += 1
net[0].weight.data[0,0] = 42
net[0].weight.data[0]

tensor([42.0000,  9.6202,  8.5495,  1.0000])

### 5.2.3. Tied Parameters

In [31]:
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4,8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8,1))

net(X)

tensor([[-0.1279],
        [-0.1252]], grad_fn=<AddmmBackward0>)

In [32]:
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0,0] = 100

tensor([True, True, True, True, True, True, True, True])


In [33]:
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


## 5.3. Custom Layers

### 5.3.1. Layers without Parameters

In [34]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X-X.mean()

In [35]:
layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [36]:
net = nn.Sequential(nn.Linear(8,128), CenteredLayer())

In [37]:
Y = net(torch.rand(4,8))
Y.mean()

tensor(0., grad_fn=<MeanBackward0>)

### 5.3.2. Layers with Parameters

In [38]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [42]:
linear = MyLinear(5,3)
linear.weight

Parameter containing:
tensor([[-0.3589,  1.2739, -0.8160],
        [-1.1852, -1.6066, -0.3141],
        [ 0.1864, -2.7890,  1.0304],
        [ 0.5852, -1.7966,  0.1661],
        [-2.0217, -1.1060,  0.3803]], requires_grad=True)

In [44]:
linear(torch.rand(2,5))

tensor([[0.0000, 0.0000, 1.3271],
        [0.0000, 0.0000, 0.2805]])

In [45]:
net = nn.Sequential(MyLinear(64,8), MyLinear(8,1))
net(torch.rand(2,64))

tensor([[2.1871],
        [1.2823]])

## 5.4. File I/O

### 5.4.1. Loading and Saving Tensors

In [46]:
x = torch.arange(4)
torch.save(x, 'x-file')

In [47]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [48]:
y = torch.zeros(4)
torch.save([x,y], 'x-files')
x2, y2 = torch.load('x-files')

In [49]:
mydict = {'x':x,'y':y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 5.4.2. Loading and Saving Model Parameters

In [50]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.output = nn.Linear(256,10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

In [51]:
net = MLP()
X = torch.randn(size=(2,20))
Y = net(X)

In [52]:
torch.save(net.state_dict(),'mlp.params')

In [54]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [55]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 5.5. GPUs

### 5.5.1. Computing Devices

In [ ]:

import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

torch.cuda.device_count()

def try_gpu(i = 0):
  if torch.cuda.device_count() >= i + 1:
      return torch.device(f'cuda:{i}')
  return torch.device('cpu')

def try_all_gpus():
  devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
  return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()


### 5.5.2. Tensors and GPUs

In [ ]:
x = torch.tensor([1,2,3])
x.device

#### Storage on the GPU

In [ ]:
X = torch.ones(2,3, device=try_gpu())
X

Y = torch.randn(2,3, device=try_gpu(1))
Y

Y.device

#### Copying

In [ ]:
Z = X.cpu()
print(X)
print(Z)

Y + X

Y+Z

Z.cpu() is Z

### 5.5.3. Neural Networks and GPUs

In [ ]:

net = nn.Sequential(nn.Linear(3,1))
net = net.to(device=try_gpu())

net(X)

net[0].weight.data.device